In [11]:
import requests
from bs4 import BeautifulSoup
import json
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [3]:
def get_token():
    load_dotenv('../.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content


In [4]:
def get_available_bikes():
    load_dotenv('../.env')
    token = os.environ.get("access_token")
    url = f"https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [5]:
data_bikes = get_available_bikes()
bicimad_real_time = pd.DataFrame(get_available_bikes()["data"])

Cleaning the dataframe

In [6]:
bicimad_real_time[["longitude", "latitude"]] = bicimad_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))

In [7]:
bicimad_real_time = bicimad_real_time.drop(["geofence", "integrator", "reservations_count", "tipo_estacionPBSC", "virtual_bikes", "virtual_bikes_num", "code_district", "code_suburb", "bikesGo"], axis = 1)
bicimad_real_time

,activate,address,dock_bikes,free_bases,geometry,id,light,name,no_available,number,total_bases,virtualDelete,geofenced_capacity,longitude,latitude
0,1,"Avenida del Ensanche de Vallecas, 9,",13,14,"{'type': 'Point', 'coordinates': [-3.61253028,...",2190,2,"453 - Avenida del Ensanche de Vallecas, 9",0,453,27,False,20.0,-3.612530,40.370440
1,1,"Paseo de la Chopera,33,Comunidad de Madrid España",22,1,"{'type': 'Point', 'coordinates': [-3.700399664...",2205,1,"267 - Paseo de la Chopera, 33",0,267,23,False,20.0,-3.700400,40.395000
2,1,"Paseo de la Castellana nº 122,",1,22,"{'type': 'Point', 'coordinates': [-3.6908, 40....",2224,0,150 - Castellana frente a Hermanos Pinzón,0,150,23,False,20.0,-3.690800,40.449100
3,1,"Paseo de la Castellana nº 164,Comunidad de Mad...",2,21,"{'type': 'Point', 'coordinates': [-3.6894151, ...",2225,0,157- Castellana 164,0,157,23,False,20.0,-3.689415,40.459137
4,1,"Guetaria , 84b,",19,5,"{'type': 'Point', 'coordinates': [-3.71569102,...",2226,1,395b - 395 - Guetaria 84b,0,395b,24,False,20.0,-3.715691,40.369168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,1,"Calle Segovia nº 26,",9,18,"{'type': 'Point', 'coordinates': [-3.713377023...",1572,2,166 - Segovia 26,0,166,27,False,20.0,-3.713377,40.413905
610,1,"Calle del Sorbe , 20,",16,8,"{'type': 'Point', 'coordinates': [-3.71320917,...",2070,2,"357 - Calle del Sorbe , 20",0,357,24,False,20.0,-3.713209,40.389870
611,1,"Avenida del Mediterráneo nº 19,",17,7,"{'type': 'Point', 'coordinates': [-3.675102, 4...",1478,1,76 - Puerta de Mariano de Cavia,0,76,24,False,20.0,-3.675102,40.407271
612,0,"Calle Francisco Balseiro nº 1,",0,0,"{'type': 'Point', 'coordinates': [-3.710525, 4...",1643,3,237 - Pablo Iglesias,1,237,0,False,0.0,-3.710525,40.451333


In [13]:
def nearest_bicimad_realtime(places, bicimad_real_time):
    data_list = []
    
    """ Latitud y longitud a radianes para realizar las operaciones. Luego los convierto a numpy para evitar error. """ 

    places_lat_rad = np.radians(places['location.latitude'].to_numpy())
    places_lon_rad = np.radians(places['location.longitude'].to_numpy())
    bicimad_lat_rad = np.radians(bicimad_real_time['latitude'].to_numpy())
    bicimad_lon_rad = np.radians(bicimad_real_time['longitude'].to_numpy())

    dlat = bicimad_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicimad_lon_rad[:, np.newaxis] - places_lon_rad

    """ Fórmula de Haversine para calcular las distancias. """ 
    
    a = np.sin(dlat / 2) ** 2 + np.cos(bicimad_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    """ Cálculo del índice del resultado con menor distancia """

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    """ Creación del dataframe del resultado utilizando ese índice"""

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicimad_real_time['name'].iloc[station_index]
        station_address = bicimad_real_time["address"].iloc[station_index]
        bikes_available = bicimad_real_time["dock_bikes"].iloc[station_index]
        station_id = bicimad_real_time["id"].iloc[station_index]
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"place": place, "place_address": place_address, "station_name": station, "bikes_available": bikes_available, "station_id" : station_id, "station_address": station_address,  "distance": min_distance})
        
    return pd.DataFrame(data_list)